In [182]:
url_dki = "https://www.olx.co.id/mobil-bekas_c198?filter=m_color_eq_putih_and_hitam%2Cm_transmission_eq_automatic%2Cm_year_min_2015%2Cprice_between_70000000_to_100000000&sorting=asc-price"


In [16]:
# diler | individu
# automatic | manual
# tahun 

def mobil_bekas_search(opt):
    filter = ""
    if len(opt)>0:
        conn = "%2C"
        filter = "?filter="
        for index,val in enumerate(opt[0]):
            if val=='tipe_penjual':
                val = f"m_seller_type_eq_seller-type-{opt[0][val]}"
            
            if val=='tipe_transmisi':
                val = f"m_transmission_eq_{opt[0][val]}"
            
            if val=='tahun_min':
                val = f"m_year_min_{opt[0][val]}"
            
            if val=='tahun_max':
                val = f"m_year_max_{opt[0][val]}"
            
            if val=='harga_min':
                val = f"price_min_{opt[0][val]}"
            
            if val=='harga_max':
                val = f"price_max_{opt[0][val]}"
            
            if val=='tahun_min_max':
                val = f"m_year_between_{opt[0][val]}"
            
            if val=='harga_min_max':
                val = f"price_between_{opt[0][val]}"
            
            filter = filter + conn + val
    
    url = "https://www.olx.co.id/mobil-bekas_c198" + filter
    print(url)

In [18]:
opt = [{
    'tipe_penjual':'individu', 
    'tipe_transmisi':'automatic',
    'tahun_min':'2010',
    'harga_min_max':'70000000_to_100000000'
    }]
mobil_bekas_search(opt)

https://www.olx.co.id/mobil-bekas_c198?filter=%2Cm_seller_type_eq_seller-type-individu%2Cm_transmission_eq_automatic%2Cm_year_min_2010%2Cm_price_between_70000000_to_100000000


In [28]:
opt[0]

{'tipe_penjual': 'individu', 'tipe_transmisi': 'automatic'}

In [109]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np

In [183]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

PATH = "D:\Programming\Python\Scrapping\driver_selenium\chromedriver.exe"

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument("start-maximized")
driver = webdriver.Chrome(PATH, chrome_options=options)

driver.get(url_dki)
print(driver.title)
WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "[data-aut-id=btnLoadMore]")))
while True:
    try:
        WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "[data-aut-id=btnLoadMore]"))).click()
        print("MORE button clicked")
        time.sleep(1) 
    except TimeoutException:
        break

page_source = driver.page_source
driver.quit()  

soup = bs(page_source, 'lxml')
items = soup.find_all(attrs={"data-aut-id":"itemBox"})

links = []
titles = []
prices = []
years = []
locations = []

# looping
for item in items:
    # title
    title_tag = item.find(attrs={"data-aut-id":"itemTitle"}).string.extract()
    titles.append(title_tag)

    # link
    a_tag = item.find('a').attrs['href']
    links.append(a_tag)

    # price
    price_tag = item.find(attrs={"data-aut-id":"itemPrice"}).string.extract()
    prices.append(price_tag)

    # year
    year_tag = item.find(attrs={"data-aut-id":"itemDetails"}).string.extract()
    years.append(year_tag)

    # location
    location_tag = item.find(attrs={"data-aut-id":"item-location"}).string.extract()
    locations.append(location_tag)

d = {'title':titles, 'price':prices, 'year':years, 'location':locations, 'link':links}
df = pd.DataFrame(d)
df['price'] = df['price'].str.replace('.', '').str.replace('Rp ', '').astype(int)
df['link'] = "https://www.olx.co.id" + df['link']
df = df.drop_duplicates(['link']).reset_index(drop=True)
print("Done..!")

Jual Beli Mobil Bekas Automatic 70000000 100000000 juta Murah & Cari Mobil Bekas di Indonesia - OLX.co.id
MORE button clicked
MORE button clicked
MORE button clicked
MORE button clicked
MORE button clicked
MORE button clicked
MORE button clicked
MORE button clicked
MORE button clicked
MORE button clicked
MORE button clicked
MORE button clicked
MORE button clicked
MORE button clicked
MORE button clicked
MORE button clicked
MORE button clicked
MORE button clicked
MORE button clicked
MORE button clicked
MORE button clicked
MORE button clicked
MORE button clicked
Done..!


In [184]:
pd.set_option('display.max_colwidth', -1)

# remove agya ayla karimun
remove_kaleng = df['title'].str.lower().str.contains('agya|ayla|karimun|agia')
df.loc[remove_kaleng==False].sort_values(by=['price'])

,title,price,year,location,link
24,Honda Mobilio 2019 Harga Bisa Nego,70000000,2019,"Cileungsi, Bogor Kab.",https://www.olx.co.id/item/honda-mobilio-2019-harga-bisa-nego-iid-794770053
26,Oper kredit Datsun Cross (30Jt nett),70000000,2018,"Bulak, Surabaya Kota",https://www.olx.co.id/item/oper-kredit-datsun-cross-30jt-nett-iid-793422115
3,Wagon R GS AT 2015 putih low KM,74500000,2015,"Serpong Utara, Tangerang Selatan Kota",https://www.olx.co.id/item/wagon-r-gs-at-2015-putih-low-km-iid-791614500
38,Datsun go+ warna putih tahun 2016,75000000,2016,"Cianjur, Cianjur Kab.",https://www.olx.co.id/item/datsun-go-warna-putih-tahun-2016-iid-790995617
36,Mobil BR-V KM Rendah,75000000,2017,"Ciampea, Bogor Kab.",https://www.olx.co.id/item/mobil-br-v-km-rendah-iid-792519559
...,...,...,...,...,...
413,Nissan march 2016 metic,100000000,2016,"Mamajang, Makassar Kota",https://www.olx.co.id/item/nissan-march-2016-metic-iid-793774450
414,Dijual Sigra 1.2 R,100000000,2017,"Pesanggrahan, Jakarta Selatan",https://www.olx.co.id/item/dijual-sigra-12-r-iid-793589392
415,Take over honda brio tangan pertama,100000000,2015,"Kelapa Dua, Tangerang Kab.",https://www.olx.co.id/item/take-over-honda-brio-tangan-pertama-iid-793576589
406,"Over Kredit Datsun Go, Terima TT",100000000,2019,"Sukajadi, Bandung Kota",https://www.olx.co.id/item/over-kredit-datsun-go-terima-tt-iid-794635327
